In [4]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np

ODIR_PATH = '../ODIR-5K/ODIR-5K/Training Images/'
BRSET_PATH = '../a-brazilian-multilabel-ophthalmological-dataset-brset-1.0.1/a-brazilian-multilabel-ophthalmological-dataset-brset-1.0.1/fundus_photos/'

def find_img_path(image_id, data_source):
    if data_source == 'ODIR':
        path = os.path.join(ODIR_PATH, image_id)
    else:
        path = os.path.join(BRSET_PATH, image_id)
    return path

def crop_image_to_circle(image):

    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    lower = np.array([3,3,3])
    higher = np.array([254,254,254])
    mask=cv2.inRange(img, lower, higher)
    
    
    contours, _ = cv2.findContours(image= mask,
                                   mode=cv2.RETR_EXTERNAL,
                                   method=cv2.CHAIN_APPROX_NONE)

    all_contours = np.vstack(contours)
    x, y, w, h = cv2.boundingRect(all_contours)  
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 5)
    cropped_img = image[y:y + h + 1, x:x + w + 1]
    return cropped_img
    
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # _, threshold = cv2.threshold(gray, None, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # if contours:
    #     largest_contour = max(contours, key=cv2.contourArea)
    #     x, y, w, h = cv2.boundingRect(largest_contour)
    #     cropped_image = image[y:y + h, x:x + w]
    #     return cropped_image
    # return False


def open_img(directory_path, input_filename):
    image_path = os.path.join(directory_path, input_filename)
    image = cv2.imread(image_path)
    return image

def preprocess(image, size_x=224, size_y=224):
    # cropping
    final_image = crop_image_to_circle(image)
    
    # blurring for reducing noise
    final_image = cv2.bilateralFilter(final_image, d=5, sigmaColor=75, sigmaSpace=75)
    # CLACHE in green channel
    green_channel = final_image[:, :, 1]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_green_channel = clahe.apply(green_channel)
    final_image[:, :, 1] = enhanced_green_channel
    
    #resizing
    final_image = cv2.resize(final_image, (size_x, size_y), interpolation=cv2.INTER_LINEAR)
    
    # # saving
    # if not os.path.exists(output_path):
    #     os.makedirs(output_path)
    # cv2.imwrite(os.path.join(output_path, output_filename))
    return final_image

def save_pic(image, output_filename, output_path="preprocessed_images"):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    cv2.imwrite(os.path.join(output_path, output_filename), image)